In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [1]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("sealuzh/app_reviews")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Convert to DataFrame and Preprocess

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json

In [5]:
# Convert the dataset to pandas DataFrame
df = pd.DataFrame(dataset['train'])
df.head()

,package_name,review,date,star
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5
3,com.mantz_it.rfanalyzer,The bandwidth seemed to be limited to maximum ...,July 25 2016,3
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5


In [29]:
df.columns = df.columns.str.replace(' ', '')

In [11]:
from datasets import load_dataset, DatasetDict

In [ ]:
# Remove duplicates
df = dataset.drop_duplicates(subset=["Review"], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into train, validation, and test sets
train_val, test = train_test_split(dataset, test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

In [17]:
# Shuffling the training data (optional, but recommended)
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [18]:
# Creating the OpenAI JSONL format
import json

def convert_to_openai_jsonl(data, file_path):
    with open(file_path, "w") as f:
        for index, row in data.iterrows():
            example = {
                "prompt": row["Review"],
                "completion": f" {row['Star']}\n\n###\n\n"
            }
            json.dump(example, f)
            f.write("\n")

In [ ]:
# Convert train, val, and test sets to OpenAI JSONL format
convert_to_openai_jsonl(train, "train.jsonl")
convert_to_openai_jsonl(val, "val.jsonl")
convert_to_openai_jsonl(test, "test.jsonl")

In [ ]:
# Save train and validation data in JSONL format
create_jsonl_format(train_data, 'train.jsonl')
create_jsonl_format(val_data, 'validation.jsonl')

In [ ]:
!pip install openai

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "your-api-key"

# Upload train, val, and test files
train_file = openai.File.create(file=open("train.jsonl"), purpose="fine_tuning")
val_file = openai.File.create(file=open("val.jsonl"), purpose="validation")
test_file = openai.File.create(file=open("test.jsonl"), purpose="testing")

In [ ]:
from openai import API

# Initialize OpenAI API client
api = API(api_key='YOUR_API_KEY')

# Upload train and validation files
train_file_id = api.upload_file('train.jsonl')
val_file_id = api.upload_file('validation.jsonl')

In [ ]:
# Start a fine-tuning job
job = openai.FineTune.create(
    model="gpt-3.5-turbo",
    training_file=train_file.id,
    suffix="my_fine_tuned_model"
)

In [ ]:
# Start a fine-tuning job
model = 'gpt-3.5-turbo'  # Choose the model to fine-tune
training_file = train_file_id  # File ID returned when uploading the training file
suffix = 'sentiment-classifier'  # Customize fine-tuned model's name
api.create_fine_tuning_job(model=model, training_file=training_file, suffix=suffix)